# An example on covariate and phenotype generation

This example shows how to generate covariates and phenotypes out of these covariates. In the process generation, covariates such as age, sex, diagnosis are considered to be independent. Estimated total intracranial volume (eTIV or ICV) is known to be dependent on age and sex. However, we won't consider its influence on diagnosis.

Data sample distribution can be iid (independent and identically distributed), mildly dependent, or strongly dependent (non-IID).

